In [28]:
import monai
from monai.metrics import DiceMetric, MeanIoU
import torch
import numpy as np
import instruction_ViT
from torch.utils.data import Dataset,DataLoader

text_features = np.random.rand(21,512)

model = timm.create_model('vit_base_patch16_224_seg',pretrained=True,num_classes=text_features.shape[0])

loss_fn=monai.losses.DiceLoss(softmax=True)
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
IoU_metric = MeanIoU(include_background=True, reduction="mean", get_not_nans=False)

class MyDataset_val(Dataset):
    def __init__(self):
        self.data=np.random.rand(100,3,224,224)
        self.label=np.random.randint(0,2,(100,21,224,224))
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image= torch.tensor(self.data[idx],dtype=torch.float)
        label = torch.tensor(self.label[idx],dtype=torch.float)
        return image,label
dataset = MyDataset_val()
val_dataloader=DataLoader(dataset,batch_size=32,drop_last=True)

def change_output(output):
    pred_y = output.cpu()
    temp=torch.argmax(pred_y,axis=1)
    temp=temp.unsqueeze(1)
    pred_y = [temp==i for i in range(pred_y.shape[1])]
    pred_y = torch.concat(pred_y,dim=1)
    return pred_y.cuda(non_blocking=True)

for images,target in val_dataloader:
    output=model(images)#input (B,3,224,224) ouput(B,segment Cannel,224,224)
    loss = loss_fn(output,target)
    ypred=change_output(output)
    target = target.cuda()
    dice_metric(y_pred=ypred, y=target)
    IoU_metric(y_pred=ypred, y=target)
metric = dice_metric.aggregate().item()
iou = IoU_metric.aggregate().item()
dice_metric.reset()
IoU_metric.reset()
print(metric)
print(iou)

0.08508483320474625
0.04491381347179413
